## 生成一个csv文件

In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

## 使用pandas读取csv格式数据

In [3]:
import pandas as pd

data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


## 分出特征矩阵inputs，标签target，并处理缺失值

In [17]:
# 各行的第一列、第二列作为输入的features， 最后一列是输出目标target
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2] 

# 处理缺失值：独热编码方式， dummy_na=True 是将nan也看作一类
inputs = pd.get_dummies(inputs, dummy_na=True)
# 将每列nan换成列的均值
inputs = inputs.fillna(inputs.mean())
inputs  #  pandas.core.frame.DataFrame

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


## 将ndarray类型的数据转换成Tensor

In [11]:
import torch
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

### problem 1 删除缺失值最多的列

In [54]:
import pandas as pd

data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [21]:
# 当前列的缺失值个数
data['NumRooms'].isna().sum()

2

In [23]:
col_name = ['NumRooms','Alley','Price']
count_max = 0
max_nan_col = ''

for name in col_name:
    count = data[name].isna().sum() # 当前列的缺失值个数
    if count > count_max:
        count_max = count
        max_nan_col = name
        
data_new = data.drop(max_nan_col,axis=1)
data_new

,NumRooms,Price
0,NaN,127500
1,2.0,106000
2,4.0,178100
3,NaN,140000


In [64]:
# 处理缺失值（平均值填充）
inputs, outputs = data_new.iloc[:, 0], data_new.iloc[:, 1]
inputs = inputs.fillna(inputs.mean()) # inputs.mean()能够计算出每列的均值
inputs

0    3.0
1    2.0
2    4.0
3    3.0
Name: NumRooms, dtype: float64

In [63]:
# 转成tensor
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([3., 2., 4., 3.], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))